# Discrete Derham complex

The basics of the 3d Derham diagram are explained in the [struphy documentation](https://struphy.pages.mpcdf.de/struphy/sections/discretization.html).

![hi](../doc/pics/derham_complex.png)

The discrete complex in the above diagram (lower row) is loaded via the **Derham class**: 

In [ ]:
import yaml
from mpi4py import MPI

import struphy
from struphy.psydac_api.psydac_derham import Derham

# use default input parameter file
path = struphy.__path__[0]
with open(path + '/io/inp/parameters.yml') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)

Nel      = params['grid']['Nel']        # Number of grid cells
p        = params['grid']['p']          # spline degrees
spl_kind = params['grid']['spl_kind']   # Spline types (clamped vs. periodic)

comm = MPI.COMM_WORLD
derham = Derham(Nel, p, spl_kind, comm=comm)

*Task 1:* Find out the number of elements, spline degrees and boundary conditions of the 1d spline spaces building up the tensor product bases.

In [ ]:
derham.Nel

In [ ]:
derham.p

In [ ]:
derham.spl_kind

*Task 2*: Define a simple periodic function on the unit cube $[0, 1]^3$ and project it into $V_h^0$. Also, project its gradient into $V_h^1$.

In [ ]:
import numpy as np

fun = lambda x, y, z : np.sin(2*np.pi*z) 
fun_h = derham.P0(fun)
print('Projected function:', fun_h)

dx_fun = lambda x, y, z : 0.
dy_fun = lambda x, y, z : 0.
dz_fun = lambda x, y, z : 2*np.pi*np.cos(2*np.pi*z)
grad_h = derham.P1((dx_fun, dy_fun, dz_fun))
print('Projected gradient:', grad_h)

*Task 3:* Get the FE coefficients of the projected function and apply the discrete gradient operator.

In [ ]:
coeffs_fun = fun_h.coeffs
print('Coefficients of function', type(coeffs_fun))

coeffs_grad = derham.grad.dot(coeffs_fun)
coeffs_grad.update_ghost_regions() # TODO: should be built-in
print('Coefficients after grad operator', type(coeffs_grad))

*Task 4:* Check the commuting property.

In [ ]:
assert np.allclose(grad_h.coeffs[0]._data, coeffs_grad[0]._data)
assert np.allclose(grad_h.coeffs[1]._data, coeffs_grad[1]._data)
assert np.allclose(grad_h.coeffs[2]._data, coeffs_grad[2]._data)

### Other Derham object properties:

In [ ]:
# Derham discretization attributes
print(f'derham.bc: {Derham.bc.__doc__} {derham.bc}\n')
print(f'derham.quad_order: {Derham.quad_order.__doc__} {derham.quad_order}\n')
print(f'derham.nq_pr: {Derham.nq_pr.__doc__} {derham.nq_pr}')

In [ ]:
# Derham misc attributes
print(f'derham.der_as_mat: {Derham.der_as_mat.__doc__} {derham.der_as_mat}\n')
print(f'derham.breaks: {Derham.breaks.__doc__} {derham.breaks}\n')

In [ ]:
# Derham global spline index attributes
print(f'derham.indN: {Derham.indN.__doc__} {derham.indN}\n')
print(f'derham.indD: {Derham.indD.__doc__} {derham.indD}')

In [ ]:
# Derham parallel attributes
print(f'derham.comm: {Derham.comm.__doc__} {derham.comm}\n')
print(f'derham.domain_array: {Derham.domain_array.__doc__} {derham.domain_array}\n')
print(f'derham.index_array_N: {Derham.index_array_N.__doc__} {derham.index_array_N}\n')
print(f'derham.index_array_D: {Derham.index_array_D.__doc__} {derham.index_array_D}\n')
print(f'derham.neighbours: {Derham.neighbours.__doc__} {derham.neighbours}')